<center>
    
# R406: Applied Economic Modelling with Python

</center>

<br> <br> 

<center>

## An introduction to working with databases in Python

</center>

<br><br> 

<center>
    
## Andrey Vassilev

</center>

 

# Why databases?

A **database** is a system designed to store, organize, and efficiently
query structured data. Databases play a central role in empirical
economics, data science, and machine learning workflows.

There are a number of reasons why databases are important:

- data may be too large to fit in memory
- simple file formats (CSV, Excel, etc.) have important limitations:
  - no enforced structure or data types
  - no guarantees about consistency
  - inefficient for repeated queries on large datasets
- data often needs to be queried, filtered, and aggregated repeatedly
- data within an organization typically exists in *multiple related datasets (tables)*
  (e.g. employees, customers, orders, inventories)
- access to data often needs to be controlled and shared across users,
  with different permissions and responsibilities
- in practice, many organizations store and manage their data in databases,
  and analysts are expected to work within these existing workflows



# Main types of databases

There are many types of databases, designed for different use cases.
At a very high level, we can distinguish between:

- **Relational databases (SQL)**  
  Data is stored in tables with rows and columns, and relationships
  between tables are explicit. Examples include:
  - SQLite
  - PostgreSQL
  - MySQL
  - DuckDB

- **Analytical / column-oriented databases**  
  Optimized for fast aggregation and analysis of large datasets. Examples include Amazon Redshift and Monet DB.
  DuckDB also belongs to this category.

- **NoSQL databases**  
  Designed for flexible or unstructured data (e.g. documents,
  key–value stores, graphs). Examples include MongoDB and Redis.

In what follows, we'll focus on relational databases.


# Python and databases

Databases typically expose their functionality through *SQL* (Structured Query Language). 
We can interact with databases from Python indirectly through specialized libraries that translate to SQL.

Common components in the Python database ecosystem include:

- **Database drivers**  
  Low-level libraries that allow Python to connect to a specific database (e.g. `sqlite3`, `psycopg`, `duckdb`).

- **SQLAlchemy**  
  A widely used library that provides a common interface for working with many different relational databases. It is often used as a bridge between Python libraries and database engines.

- **High-level libraries**  
  Libraries such as Pandas build on top of these tools to provide convenient ways to read data from and write data to databases.

Here, we will interact with databases through Pandas and DuckDB, without requiring detailed knowledge of SQLAlchemy.

# Interfacing with a database from Pandas

- Pandas provides functions to retrieve data from databases and to write
  `DataFrame`s back to databases.
- This functionality relies on the `SQLAlchemy` library, which provides
  a common interface to many different database systems.
- Common functions include:
  - `read_sql_table()` to retrieve an entire table from a database,
  - `read_sql_query()` to run an SQL query against the database.
- A `DataFrame` has a method `to_sql()` to write its contents as a table
  in a database.

# Reading tables from a database with Pandas

We start by connecting to an existing SQLite database file,
`company_data.db`. This database already contains several tables.

Pandas provides the function `read_sql_table()` to load an entire table
from a database into a `DataFrame`.

In this case we happen to know in advance that the database has tables named `firms` and `products`.

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine("sqlite:///company_data.db")

firms = pd.read_sql_table("firms", engine)
products = pd.read_sql_table("products", engine)

In [ ]:
firms

In [ ]:
products

# Writing a Pandas dataframe to a database

We now combine the data from the different tables and do additional calculations:

In [ ]:
merged = firms.merge(products, on="firm_id")
merged["price_in_EUR"] = merged["price"]/1.95583
merged


It's now time to write the result back to the database:

In [ ]:
merged.to_sql("firm_products", engine, if_exists="replace", index=False)

# Running SQL queries from Pandas

We now get curious and ask ourselves if our table was successfully written to the database and what other tables are contained there. The following query asks the database which tables it contains.

In [ ]:
pd.read_sql_query(
    "SELECT name FROM sqlite_master WHERE type='table'",
    engine
)

# A quick look at DuckDB

DuckDB is a lightweight analytical database designed for fast querying and aggregation. 
Unlike “server” databases, DuckDB runs *inside* your Python process (similar to SQLite), which makes it very convenient for data work in Jupyter.

DuckDB can work seamlessly with:

- SQL queries
- Pandas DataFrames
- files (CSV / Parquet)
- Polars DataFrames (preview)

**Note:** We will not focus on SQL syntax here. 

In [ ]:
import duckdb

# In-memory DuckDB (no files created)
con = duckdb.connect()

# Quick check
con.execute("SELECT 1 AS x").fetchall()


In [ ]:
# Alternative style: connection as a context manager
with duckdb.connect() as con2:
    print(con2.execute("SELECT 2 AS y").fetchall())


# DuckDB + Pandas

DuckDB can query Pandas DataFrames directly. This is useful when you
already have data in memory but want to use SQL for certain operations
(e.g. quick aggregation or combining with data from an external source).

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine("sqlite:///company_data.db")

firms = pd.read_sql_table("firms", engine) # reload just in case
products = pd.read_sql_table("products", engine) # reload just in case
sales = pd.read_sql_table("sales", engine)

display(firms)
display(products)
display(sales)

In [ ]:
import duckdb

con = duckdb.connect()

# DuckDB automatically "sees" DataFrames referenced in the query
revenue_by_firm = con.execute("""
    SELECT
        f.firm_name,
        SUM(s.quantity * p.price) AS revenue
    FROM sales AS s
    JOIN products AS p USING (product_id)
    JOIN firms AS f USING (firm_id)
    GROUP BY f.firm_name
    ORDER BY revenue DESC
""").df()

revenue_by_firm

# DuckDB + files

DuckDB can query data stored in files directly (CSV, Parquet, etc.).
This can be useful when:

- files are large
- you only need a subset of rows/columns
- you want to filter/aggregate without loading everything into memory


In [ ]:
# Create a small CSV file for demonstration
sales.to_csv("sales.csv", index=False)

Here is how we can read it and process it in DuckDB:

In [ ]:
con = duckdb.connect()

sales_summary = con.execute("""
    SELECT year, SUM(quantity) AS total_quantity
    FROM 'sales.csv'
    GROUP BY year
    ORDER BY year
""").df()

sales_summary


Here is how we can write back to a new file directly from DuckDB:

In [ ]:
con.execute("""
    COPY (
        SELECT year, SUM(quantity) AS total_quantity
        FROM 'sales.csv'
        GROUP BY year
        ORDER BY year
    )
    TO 'sales_summary.csv' (HEADER, DELIMITER ',')
""")


# DuckDB + Polars

Polars is a fast DataFrame library similar to Pandas.

DuckDB mixes seamlessly with Polars and Pandas, which makes it easy to combine Pandas and Polars dataframe operations with SQL queries.

In [ ]:
import polars as pl

# Convert a Pandas dataframe to a Polars dataframe
sales_pl = pl.from_pandas(sales)
sales_pl


Use DuckDB to connect to our SQLite database, process and output the result as a Polars dataframe:

In [ ]:
con = duckdb.connect()

# Enable DuckDB's ability to read SQLite databases
con.execute("INSTALL sqlite_scanner;")
con.execute("LOAD sqlite_scanner;")

# Attach the SQLite database file
con.execute("ATTACH 'company_data.db' AS company_db (TYPE sqlite);")

# Query the SQLite tables using SQL
revenue_by_firm_pl = con.execute("""
    SELECT
        f.firm_name,
        SUM(s.quantity * p.price) AS revenue
    FROM company_db.sales AS s
    JOIN company_db.products AS p USING (product_id)
    JOIN company_db.firms AS f USING (firm_id)
    GROUP BY f.firm_name
    ORDER BY revenue DESC
""").pl()

revenue_by_firm_pl

In [ ]:
# Polars to Pandas
revenue_by_firm_pl.to_pandas()

Close everything to clean up (not strictly necessary in notebooks but good practice):

In [ ]:
engine.dispose()
con.close()

# Choosing the right tool

The appropriate tool to use depends on the task and the data size.

- **Pandas only**
  - data fits comfortably in memory
  - exploratory analysis, quick transformations
  - prototyping models and ideas


- **Databases + Pandas**
  - data is structured and stored persistently
  - multiple related tables
  - repeated reads and writes
  - integration with existing data workflows


- **DuckDB (often with Pandas or Polars)**
  - analytical queries and aggregations
  - working with large files (CSV, Parquet) without loading everything
  - mixing SQL queries with DataFrame-based workflows
  - larger-than-memory datasets and lazy evaluation